# Final

## Setup

In [2]:
from google.colab import drive
import sys
drive.mount('/content/drive')

nb_path = '/content/colab_libraries'
# sys.path.insert(0,nb_path)

# !pip install --target=$nb_path pyrosettacolabsetup
# !pip install --target=$nb_path py3Dmol
# !pip install --target=$nb_path nglview

# sys.path.append("/usr/local/lib/python3.9/site-packages")

import pyrosettacolabsetup; pyrosettacolabsetup.install_pyrosetta()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  Using cached pyrosettacolabsetup-1.0.9-py3-none-any.whl.metadata (294 bytes)
Using cached pyrosettacolabsetup-1.0.9-py3-none-any.whl (4.9 kB)
  Using cached py3Dmol-2.4.2-py2.py3-none-any.whl.metadata (1.9 kB)
Using cached py3Dmol-2.4.2-py2.py3-none-any.whl (7.0 kB)
  Using cached nglview-3.1.4-py3-none-any.whl
  Using cached ipywidgets-8.1.5-py3-none-any.whl.metadata (2.3 kB)
  Using cached notebook-7.3.2-py3-none-any.whl.metadata (10 kB)
  Using cached jupyterlab-4.3.4-py3-none-any.whl.metadata (16 kB)
  Using cached jupyterlab_widgets-3.0.13-py3-none-any.whl.metadata (4.1 kB)
  Using cached numpy-2.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached comm-0.2.2-py3-none-any.whl.metadata (3.7 kB)
  Using cached ipython-8.31.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached traitlets-5.14.3-py3-none-any.whl.metad

In [3]:
import pyrosetta; pyrosetta.init()
from pyrosetta import *
import glob
print(glob.glob("*.pdb"))

┌──────────────────────────────────────────────────────────────────────────────┐
│                                 PyRosetta-4                                  │
│              Created in JHU by Sergey Lyskov and PyRosetta Team              │
│              (C) Copyright Rosetta Commons Member Institutions               │
│                                                                              │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE │
│         See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└──────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2025 [Rosetta PyRosetta4.MinSizeRel.python311.ubuntu 2025.03+release.1f5080a079a5261122c0e532c46f61a4f7e20df8 2025-01-13T15:37:50] retrieved from: http://www.pyrosetta.org
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.MinSizeRel.python311.ubuntu r390 2025.03+release.1f5080a079 1f50

In [4]:
from pyrosetta.toolbox import cleanATOM, mutate_residue
from random import randrange,choice
import copy
AA= ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']
sfxn = get_score_function()
fr = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn_in=sfxn , standard_repeats=1)# , standard_repeats=1
# fr.constrain_relax_segments()

core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBFadeIntervals.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBEval.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/DonStrength.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/AccStrength.csv
core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard res

## Prepare the pdb files for AIP and the DARPin

Use [this link](https://direvo.mutanalyst.com/params) to apply [this github repo](https://github.com/matteoferla/rdkit_to_params) to the AIP .mol file created manually based on [this paper](https://www.pnas.org/doi/10.1073/pnas.1506030112) and exported from pymol. This provides a .params file that is needed for pyrosetta to interpret this molecule not as a standard protein, but as a ligand small molecule (because otherwise it doesn't like the non-standard cyclic structure).

In [ ]:
ligand_params = Vector1(['drive/MyDrive/BME406team/Flex-ddG/Autoinduced_Peptide_1.params'])

# added a line to the params file to ensure that
# it allows sampling from all 10 rotamers of AIP during the FastRelax protocol
# these 10 rotamers are not in the base file, but in the rotamers file

pose = pyrosetta.Pose()

# code from https://graylab.jhu.edu/pyrosetta/downloads/scripts/demo/D120_Ligand_interface.py

res_set = pose.conformation().modifiable_residue_type_set_for_conf()
res_set.read_files_for_base_residue_types( ligand_params )
pose.conformation().reset_residue_type_set_for_conf( res_set )

pyrosetta.pose_from_file(pose, "drive/MyDrive/BME406team/Flex-ddG/Autoinduced_Peptide_1_base.pdb")

print(sfxn(pose))
fr.apply(pose)
print(sfxn(pose)) # it chooses the lowest energy rotamer for now!

# chain_pose=pyrosetta.rosetta.core.pose.Pose.split_by_chain(pose)

# print(len(chain_pose))
# a=[i for i in chain_pose]
# [print(sfxn(a[i])) for i in range(10)]

# a[6].dump_pdb("AIP_best.pdb")

pose.dump_pdb("drive/MyDrive/BME406team/Flex-ddG/AIP_best.pdb")

core.conformation.Conformation: [ WARNING ] Attempted to determine the residue type set of an empty pose.
core.import_pose.import_pose: File 'drive/MyDrive/BME406team/Flex-ddG/Autoinduced_Peptide_1_base.pdb' automatically determined to be of type PDB
core.io.pdb.HeaderInformation: [ ERROR ] Malformed Compound record found: ' AIP'
27.7766784486412
protocols.relax.FastRelax: CMD: repeat  27.7767  nan  nan  0.55
protocols.relax.FastRelax: CMD: coord_cst_weight  27.7767  nan  nan  0.55
protocols.relax.FastRelax: CMD: scale:fa_rep  27.7767  nan  nan  0.022
core.pack.task: Packer task: initialize from command line()
core.pack.rotamers.SingleLigandRotamerLibrary: Added 10 rotamers for AIP
core.pack.pack_rotamers: built 11 rotamers at 1 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
protocols.relax.FastRelax: CMD: repack  10.9479  nan  nan  0.022
protocols.relax.FastRelax: CMD: scale:fa_rep  10.9479  nan  nan  0.02805
protocols.relax.Fas

True

Alphafolded the DARPin sequence, converted the .cif file to a .pdb file used here

In [29]:
# binderc=pose_from_file('AIP-I-please.cif', read_fold_tree=True, type=pyrosetta.rosetta.core.import_pose.FileType.CIF_file)
# binderc=pose_from_file('drive/MyDrive/BME406team/Flex-ddG/DARPin_fold.cif', read_fold_tree=True, type=pyrosetta.rosetta.core.import_pose.FileType.CIF_file)
binderc=pose_from_pdb('drive/MyDrive/BME406team/Flex-ddG/DARPin_fold.pdb')
print(sfxn(binderc))
fr.apply(binderc)
print(sfxn(binderc))
fr.apply(binderc)
print(sfxn(binderc))
fr.apply(binderc)
print(sfxn(binderc))
binderc.dump_pdb("drive/MyDrive/BME406team/Flex-ddG/DARPin_best.pdb")

-428.30518376849
-587.8905386792311
-588.9119268963791
-588.769925975024


True

## Full Flex-DDG from iGEM DARPin

### Combine PDB files and relax

Used pymol to manually combine the DARPin with AIP in 5 different initial configurations. Try relaxing all and see which gets lowest energy? Then assume that structure is how it normally binds, and optimize from there

In [33]:
energies=[[0,0,0,0] for i in range(5)]

ligand_params = Vector1(['drive/MyDrive/BME406team/Flex-ddG/Autoinduced_Peptide_1.params'])
pose = pyrosetta.Pose()
res_set = pose.conformation().modifiable_residue_type_set_for_conf()
res_set.read_files_for_base_residue_types( ligand_params )
pose.conformation().reset_residue_type_set_for_conf( res_set )
for i in range(5):
    pose_from_file(pose, "drive/MyDrive/BME406team/Flex-ddG/Binder+AIP_options/AIP_Binder_"+str(i+1)+".pdb")
    energies[i][0]=sfxn(pose)
    for j in range(3):
        fr.apply(pose)
        energies[i][j+1]=sfxn(pose)
        print(energies)

[[524.407356063892, -583.7837593817004, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
[[524.407356063892, -583.7837593817004, -584.6667501454315, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
[[524.407356063892, -583.7837593817004, -584.6667501454315, -585.2322315222666], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
[[524.407356063892, -583.7837593817004, -584.6667501454315, -585.2322315222666], [6554.6713487588895, -579.1082024120345, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
[[524.407356063892, -583.7837593817004, -584.6667501454315, -585.2322315222666], [6554.6713487588895, -579.1082024120345, -583.0821768382589, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
[[524.407356063892, -583.7837593817004, -584.6667501454315, -585.2322315222666], [6554.6713487588895, -579.1082024120345, -583.0821768382589, -586.5125765283539], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
[[524.407356063892, -583.7837593817004, -584.6667501454315, -585.23223152226

Output of the above:

[[524.407356063892, -583.7837593817004, -584.6667501454315, -585.2322315222666],

[6554.6713487588895, -579.1082024120345, -583.0821768382589, -586.5125765283539],

[4496.01379091503, -570.1285169729424, -570.7696324307642, -578.8451111521164],

[707.8885717434343, -584.3425479140429, -584.7510178103424, -581.0737362773634],

[-577.8436817189021, -578.1326669929686, -576.7729430403792, -577.2334408723765]]

In [34]:
ligand_params = Vector1(['drive/MyDrive/BME406team/Flex-ddG/Autoinduced_Peptide_1.params'])
pose = pyrosetta.Pose()
res_set = pose.conformation().modifiable_residue_type_set_for_conf()
res_set.read_files_for_base_residue_types( ligand_params )
pose.conformation().reset_residue_type_set_for_conf( res_set )
pose_from_file(pose, "drive/MyDrive/BME406team/Flex-ddG/Binder+AIP_options/AIP_Binder_2.pdb")
print(sfxn(pose))
fr.apply(pose)
print(sfxn(pose))
fr.apply(pose)
print(sfxn(pose))
fr.apply(pose)
print(sfxn(pose))
fr.apply(pose)
print(sfxn(pose))
pose.dump_pdb("drive/MyDrive/BME406team/Flex-ddG/Original_DARPin_Bound.pdb")

6554.6713487588895
-578.456756423737
-582.322718044388
-583.0647273226798
-585.1220404688302


True

### Flex ddG

In [ ]:
# flexddg
ligand_params = Vector1(['drive/MyDrive/BME406team/Flex-ddG/Autoinduced_Peptide_1.params'])
pose = pyrosetta.Pose()
pose_old = pyrosetta.Pose()
res_set = pose.conformation().modifiable_residue_type_set_for_conf()
res_set.read_files_for_base_residue_types( ligand_params )
pose.conformation().reset_residue_type_set_for_conf( res_set )
pose_old.conformation().reset_residue_type_set_for_conf( res_set )
for i in range(1000):
    pose_from_file(pose, "drive/MyDrive/BME406team/Flex-ddG/flexddg_run.pdb")
    mutate_residue(pose,randrange(1,len(pose.sequence())+1),choice(AA)) # 1-indexed AA to mutate
    fr.apply(pose)
    print("relaxed")
    pose_from_file(pose_old, "drive/MyDrive/BME406team/Flex-ddG/flexddg_run.pdb")
    if sfxn(pose)<sfxn(pose_old):
        pose.dump_pdb("drive/MyDrive/BME406team/Flex-ddG/flexddg_run.pdb")
        print("saved")

In [ ]:
!pip install pyrosetta-installer
!python -c 'import pyrosetta_installer; pyrosetta_installer.install_pyrosetta()'

PyRosetta install detected, doing nothing...


In [ ]:
!wget https://graylab.jhu.edu/download/PyRosetta4/archive/release/PyRosetta4.Release.python38.linux/PyRosetta4.Release.python38.linux.release-321.tar.bz2
!tar -xjf PyRosetta4.Release.python38.linux.release-321.tar.bz2
import sys
sys.path.append('/content/PyRosetta4.Release.python38.linux.release-321')
import pyrosetta
pyrosetta.init()

In [ ]:
from pyrosetta import *
from pyrosetta.rosetta.protocols.ddg import ddGMover
from pyrosetta.rosetta.protocols.relax import FastRelax
from pyrosetta.rosetta.core.scoring import ScoreFunction
from pyrosetta.rosetta.core.pose import Pose
from pyrosetta.rosetta.protocols.simple_moves import MutateResidue

# Initialize PyRosetta
pyrosetta.init()

# Load the PDB file
pose = Pose()
pyrosetta.rosetta.core.import_pose.pose_from_file(pose, "4bxi.pdb")

# Create a score function
scorefxn = pyrosetta.create_score_function("ref2015")

# Set up the ddG mover
ddg = ddGMover()
ddg.score_function(scorefxn)
ddg.num_iterations(50)  # Number of iterations for sampling, increase for better results
ddg.neighbor_cutoff(8.0)  # Angstroms

# Set up the FastRelax mover for minimization
relax = FastRelax()
relax.set_scorefxn(scorefxn)

# Function to perform flex ddG calculation
def calculate_flex_ddg(pose, mutation):
    # Create a copy of the original pose
    mutant_pose = Pose(pose)

    # Perform the mutation
    mutate = MutateResidue(mutation[1], mutation[2])
    mutate.apply(mutant_pose)

    # Relax the mutant pose
    relax.apply(mutant_pose)

    # Calculate ddG
    ddg.apply(mutant_pose)

    # Get the ddG value
    ddg_value = ddg.ddg()

    return ddg_value

# Example usage
mutation = ('A', 10, 'ALA')  # Chain A, residue 10, mutate to Alanine
ddg_value = calculate_flex_ddg(pose, mutation)
print(f"Predicted ddG for mutation {mutation}: {ddg_value} REU")

┌──────────────────────────────────────────────────────────────────────────────┐
│                                 PyRosetta-4                                  │
│              Created in JHU by Sergey Lyskov and PyRosetta Team              │
│              (C) Copyright Rosetta Commons Member Institutions               │
│                                                                              │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE │
│         See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└──────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2025 [Rosetta PyRosetta4.Release.python311.ubuntu 2025.03+release.1f5080a079a5261122c0e532c46f61a4f7e20df8 2025-01-13T15:37:50] retrieved from: http://www.pyrosetta.org
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python311.ubuntu r390 2025.03+release.1f5080a079 1f5080a079

## Rational Flex-DDG from Randomized DARPin

Used pymol to manually combine the DARPin with AIP in 8 different initial configurations, then applied the following code in a .py file on a compute cluster to run Flex-ddG multiple times on each of these 8 configurations in parallel.

In [ ]:
import pyrosetta_installer

pyrosetta_installer.install_pyrosetta()
import pyrosetta
from pyrosetta import *
import sys
from tqdm import tqdm
from pyrosetta.toolbox import cleanATOM, mutate_residue
from random import randrange, choice
import copy
from datetime import datetime

job_id = int(sys.argv[1])

pyrosetta.init("-mute all")
# pyrosetta.init("-mute core basic protocols")

AA = [
    "A",
    "R",
    "N",
    "D",
    "C",
    "Q",
    "E",
    "G",
    "H",
    "I",
    "L",
    "K",
    "M",
    "F",
    "P",
    "S",
    "T",
    "W",
    "Y",
    "V",
]
residue_mut = [[32], [33, 34, 36, 44, 45], [41], [57]]
# reduced number of possible mutations: speeds up algorithm and increases DARPin fitness
AA_options = [
    ["D", "N", "S", "T"],
    ["A", "R", "N", "D", "Q", "E", "H", "K", "S", "T", "W", "Y"],
    ["A", "S", "T", "V", "L"],
    ["N", "H", "Y"],
]
all_mutations = [[]]
for i in range(len(residue_mut)):
    for j in range(len(residue_mut[i])):
        for k in range(len(AA_options[i])):
            for l in range(3):
                all_mutations.append([residue_mut[i][j] + 1 + l * 33, AA_options[i][k]])
all_mutations = all_mutations[1:]

sfxn = get_score_function()
fr = pyrosetta.rosetta.protocols.relax.FastRelax(
    scorefxn_in=sfxn, standard_repeats=1
)  # , standard_repeats=1
# fr.constrain_relax_segments()

ligand_params = Vector1(["./Autoinduced_Peptide_1.params"])
pose = pyrosetta.Pose()
pose_old = pyrosetta.Pose()
res_set = pose.conformation().modifiable_residue_type_set_for_conf()
res_set.read_files_for_base_residue_types(ligand_params)
pose.conformation().reset_residue_type_set_for_conf(res_set)
pose_old.conformation().reset_residue_type_set_for_conf(res_set)

pose_from_file(pose, "./input/DARPin_Bound_" + str(job_id // 5 + 1) + ".pdb")
for i in range(10000):  # Make a lot of the allowed mutations to randomize the sequence to start with
    mut = choice(all_mutations)
    mutate_residue(pose, mut[0], mut[1])  # 1-indexed AA to mutate

fr.apply(pose)

pose.dump_pdb("./output/flex_" + str(job_id) + ".pdb")

print("starting loop:<" + str(sfxn(pose)) + "," + str(pose.sequence()) + ">")
start_time = datetime.now()
for i in range(100000):
    pose_from_file(pose, "./output/flex_" + str(job_id) + ".pdb")
    mut = choice(all_mutations)
    mutate_residue(pose, mut[0], mut[1])  # 1-indexed AA to mutate
    fr.apply(pose)
    pose_from_file(pose_old, "./output/flex_" + str(job_id) + ".pdb")
    if sfxn(pose) < sfxn(pose_old):
        pose.dump_pdb("./output/flex_" + str(job_id) + ".pdb")
        print(
            "["
            + str(datetime.now() - start_time)
            + "]saved:<"
            + str(sfxn(pose))
            + ","
            + str(pose.sequence())
            + ">"
        )
